# Vektorisierung

In [1]:
import pandas as pd

Dataset "train_cleaned" importieren

In [2]:
df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)

In [3]:
df.head()

,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
id,,,,,,
1,0,@user when a father is dysfunctional and is s...,father selfish drag kid run,1,['#run'],NaN
2,0,@user @user thanks for #lyft credit i can't us...,thank lyft credit use cause offer van,2,"['#lyft', '#disapointed', '#getthanked']",NaN
3,0,bihday your majesty,bihday majesty,0,[],NaN
4,0,#model i love u take with u all the time in ...,model take time mobile phone kiss sunglass mou...,0,['#model'],":mobile_phone:,:kissing_face_with_smiling_eyes..."
5,0,factsguide: society now #motivation,factsguide society motivation,0,['#motivation'],NaN


Da es leere tweet_cleaned gibt müssen diese hier entfernt werden. Ansonsten werfen alle 3 Methoden Fehler

In [4]:
df = df[df['tweet_cleaned'].notna()]

## Bag of Words

In [5]:
# Doku: https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() # CountVectorizer(binary=True) würde nur 0 und 1 ausgeben
X_bow = vectorizer.fit_transform(df.tweet_cleaned)

In [6]:
print(vectorizer.get_feature_names_out())

['aamp' 'aap' 'ab' ... 'zootopia' 'zoro' 'zzz']


In [7]:
print(X_bow.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
# Erster Test NB

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

y = df.label
clf = MultinomialNB()

clf.fit(X_bow,y)

ypred = clf.predict(X_bow)

print("Accuracy: ", accuracy_score(y, ypred))

Accuracy:  0.9523671742196824


#### Ergebnis Bag of Word in X_bow

## TF-IDF

In [11]:
# Doku https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df.tweet_cleaned)

In [12]:
print(tfidf_vectorizer.get_feature_names_out())

['aamp' 'aap' 'ab' ... 'zootopia' 'zoro' 'zzz']


In [13]:
print(X_tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# Erster Test NB

y = df.label
clf = MultinomialNB()

clf.fit(X_tfidf,y)

ypred_tfidf = clf.predict(X_tfidf)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

Accuracy:  0.954609569966364


#### Ergebnis TF-IDF in X_tfidf

## Word2Vec

In [5]:
from nltk import word_tokenize
from gensim.models import Word2Vec
import numpy as np

data = df['tweet_cleaned'].map(word_tokenize)

In [6]:
data

id
1                        [father, selfish, drag, kid, run]
2            [thank, lyft, credit, use, cause, offer, van]
3                                        [bihday, majesty]
4        [model, take, time, mobile, phone, kiss, sungl...
5                        [factsguide, society, motivation]
                               ...                        
31956    [less, week, grin, sweat, fold, hand, medium, ...
31957         [fishing, tomorrow, wait, first, time, year]
31958                                    [eat, youuu, red]
31959    [see, turner, try, wrap, genuine, hero, like, ...
31960    [listen, sad, song, monday, morning, otw, work...
Name: tweet_cleaned, Length: 27649, dtype: object

In [7]:
w2v = Word2Vec(data,window=4, min_count=1, sg=0) 
# sg=1: Skip-Gram
# sg=0: Continuous Bag of Words

In [8]:
w2v.train(data, total_examples=len(df['tweet_cleaned']), epochs=20)
# epochs = Anzahl Iterationen

(3895247, 4126720)

In [9]:
w2v.wv.most_similar(positive="motivation")

[('inspiration', 0.8637232184410095),
 ('positivity', 0.8471518754959106),
 ('inspirational', 0.8018321990966797),
 ('bestseller', 0.7903605103492737),
 ('quoteoftheday', 0.7853137850761414),
 ('belief', 0.7619601488113403),
 ('entrepreneur', 0.7595241665840149),
 ('factsguide', 0.7569527626037598),
 ('motivate', 0.7558882832527161),
 ('positivevibe', 0.7505861520767212)]

In [10]:
w2v.wv.most_similar(negative="motivation")

[('meet', 0.3771251440048218),
 ('august', 0.352364182472229),
 ('july', 0.34603098034858704),
 ('bus', 0.3416936993598938),
 ('driver', 0.33455905318260193),
 ('release', 0.3304154574871063),
 ('st', 0.32495012879371643),
 ('th', 0.32050198316574097),
 ('yr', 0.3202992081642151),
 ('rd', 0.31380757689476013)]

In [11]:
size = 100

def w2v_vector(tokenized_tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokenized_tweet:
        try:
            vec += w2v.wv[word].reshape((1, size))
            count += 1
        except KeyError:
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [12]:
X_w2v = np.zeros((len(data),size))

In [13]:
for i in range(len(data)):
    X_w2v[i,:] = w2v_vector(data.iloc[i],size)

In [14]:
X_w2v

array([[ 0.03039333,  0.14097588, -0.40483578, ..., -1.17256047,
        -0.42760487, -0.77401052],
       [-0.46381379,  0.53667628,  0.16405297, ..., -0.27580128,
        -0.1997893 , -0.49546429],
       [ 0.08214973,  0.24564286, -0.4936442 , ..., -0.20577175,
         0.18469378,  0.70640232],
       ...,
       [-0.6783981 , -0.73566437,  0.89766298, ..., -0.63281615,
         0.49310272, -0.25537541],
       [-0.3309418 ,  0.46646499, -0.08161603, ..., -0.17433416,
        -0.06822646, -0.09428725],
       [-0.65280033,  0.43611948, -0.79287506, ...,  0.21845217,
         0.28302161,  0.13811521]])

In [15]:
# Erster Test NB
from sklearn.naive_bayes import GaussianNB

y = df.label
clf = GaussianNB()

clf.fit(X_w2v,y)

ypred_tfidf = clf.predict(X_w2v)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

NameError: name 'accuracy_score' is not defined

#### Ergebnis Word2Vec in X_w2v

## FastText

In [16]:
from gensim.models import FastText 

ft = FastText(data, window = 4)
ft.train(data, total_examples=len(df['tweet_cleaned']), epochs=20)

(3789191, 4126720)

In [17]:
ft.wv.most_similar(positive="motivation")

[('motivational', 0.9571335315704346),
 ('mondaymotivation', 0.9560210704803467),
 ('tuesdaymotivation', 0.9544017910957336),
 ('inspiration', 0.8639123439788818),
 ('meditation', 0.8313320279121399),
 ('affirmation', 0.807649552822113),
 ('transformation', 0.8043513894081116),
 ('innovation', 0.7998110055923462),
 ('inspirational', 0.7940093278884888),
 ('motivated', 0.7821134924888611)]

In [32]:
ft.wv.most_similar(negative="motivation")

[('yr', 0.3835121691226959),
 ('v', 0.3463435471057892),
 ('hi', 0.34137162566185),
 ('gator', 0.3400598168373108),
 ('sit', 0.32863670587539673),
 ('cox', 0.3283883035182953),
 ('xbox', 0.3270142078399658),
 ('die', 0.32616955041885376),
 ('snatch', 0.3259742558002472),
 ('bro', 0.32236719131469727)]

In [33]:
size = 100

def ft_vector(tokenized_tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokenized_tweet:
        try:
            vec += ft.wv[word].reshape((1, size))
            count += 1
        except KeyError: 
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [34]:
X_ft = np.zeros((len(data),size))

In [35]:
for i in range(len(data)):
    X_ft[i,:] = ft_vector(data.iloc[i],size)

In [36]:
X_ft

array([[-0.76547772, -0.21383475, -0.40605466, ..., -0.95944892,
        -0.78261749,  0.35719954],
       [-0.5774988 , -0.04261827, -0.24013078, ..., -0.04634736,
         0.61478294,  0.17527176],
       [ 0.81800085,  0.57611634,  0.08267095, ..., -1.20095155,
        -0.38159819, -0.06786457],
       ...,
       [-0.16428977, -0.29616243,  0.1838846 , ..., -0.87193196,
         0.509852  , -0.64583018],
       [-0.16578822, -0.05996855, -0.39674566, ...,  0.05825424,
         0.52146263,  0.31040981],
       [-0.51245216,  0.18201927, -0.49733237, ..., -0.3617763 ,
        -0.34870465, -0.19760916]])

In [37]:
# Erster Test NB
from sklearn.naive_bayes import GaussianNB

y = df.label
clf = GaussianNB()

clf.fit(X_ft,y)

ypred_tfidf = clf.predict(X_ft)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

Accuracy:  0.7548555101450324


#### Ergebnis FastText in X_ft